In [1]:
# import libraries

import os # file handling
import numpy as np # linear algebra
import nibabel as nb # neuroimaging file handling
import glob # file handling
from scipy.ndimage import zoom # image processing
import gc # garbage collection

cleaned_data_path = 'cleaned_data/'
resampled_data_path = 'resampled_data/'

if not os.path.exists(resampled_data_path):
    os.makedirs(resampled_data_path)

# get target pixel dimension
target_resolution = np.array((1.0, 1.0, 1.0)) * 0.8 # in microns

# start cleaning
print("NIBABEL VERSION:", nb.__version__)
print(f"TARGET RESOLUTION: {target_resolution[0]} μm x {target_resolution[1]} μm x {target_resolution[2]} μm")

NIBABEL VERSION: 5.1.0
TARGET RESOLUTION: 0.8 μm x 0.8 μm x 0.8 μm


In [2]:
# load data from cleaned_data folder
data_files = glob.glob(cleaned_data_path + '*.nii.gz')

# set index for data_files
index = 0
full_data = nb.load(data_files[index])
print(f"Data file: {data_files[index]}")

# get pixel dimensions in microns (x, y, z)
assert full_data.header.get_xyzt_units()[0] == 'micron', "Pixel dimensions are not in microns"
pixel_dims = full_data.header['pixdim'][1:4]
print(f"Pixel dimensions: {pixel_dims[0]} μm x {pixel_dims[1]} μm x {pixel_dims[2]} μm")

# get image dims (x, y, z)
image_dims = full_data.header['dim'][1:4]
print(f"Image dimensions: {image_dims[0]} x {image_dims[1]} pixels x {image_dims[2]} slices, {np.prod(image_dims)} voxels")


# get resampling factor for each dimension
resampling_factor = np.divide(pixel_dims, target_resolution)
# add a singleton dimension to account for the channel dimension
resampling_factor = np.append(resampling_factor, 1)
print(f"Resampling factor: {resampling_factor[0]:.2f} x {resampling_factor[1]:.2f} x {resampling_factor[2]:.2f}")

Data file: cleaned_data/synA647_LL_L3_200724.nii.gz
Pixel dimensions: 0.1289733499288559 μm x 0.1289733499288559 μm x 0.29493609070777893 μm
Image dimensions: 3048 x 3046 pixels x 600 slices, 5570524800 voxels
Resampling factor: 0.16 x 0.16 x 0.37


In [3]:
# get data in dtype in int16
data_array = full_data.get_fdata(dtype=np.float32)

In [4]:
# resample data using scipy bilinear interpolation
resampled_data_array = zoom(data_array, resampling_factor, order=1)

In [5]:
# get new image dimensions
new_image_dims = resampled_data_array.shape[0:3]
print(f"New image dimensions: {new_image_dims[0]} x {new_image_dims[1]} pixels x {new_image_dims[2]} slices, {np.prod(new_image_dims)} voxels")

# get new pixel dimensions
new_pixel_dims = np.multiply(pixel_dims, np.divide(image_dims, new_image_dims))
print(f"New pixel dimensions: {new_pixel_dims[0]:.2f} μm x {new_pixel_dims[1]:.2f} μm x {new_pixel_dims[2]:.2f} μm")

New image dimensions: 491 x 491 pixels x 221 slices, 53278901 voxels
New pixel dimensions: 0.80 μm x 0.80 μm x 0.80 μm


In [6]:
# normalize data to [0, 1]
resampled_data_array = (resampled_data_array - np.min(resampled_data_array)) / (np.max(resampled_data_array) - np.min(resampled_data_array))

# convert to int16
resampled_data_array = (resampled_data_array * 65535).astype(np.uint16)

In [7]:
# define as nifti object

# new affine
new_affine = np.copy(full_data.affine)
# modify affine to reflect new pixel dimensions
new_affine = new_affine @ np.diag(np.append(new_pixel_dims, 1.0))

# new header
new_header = full_data.header.copy()
# modify header to reflect new dimensions
new_header['pixdim'][1:4] = target_resolution
new_header['dim'][1:4] = data_array.shape[0:3]

# save resampled data
resampled_data = nb.Nifti1Image(resampled_data_array, new_affine, new_header)
nb.save(resampled_data, resampled_data_path + f"resampled_{index}.nii.gz")

# # initialize garbage collection
# del resampled_data_array, data_array, resampled_data, full_data, new_affine, new_header, new_image_dims, new_pixel_dims, pixel_dims, resampling_factor, target_resolution, data_files

# # force garbage collection
# import gc
# gc.collect()


In [10]:
#view resampled data with a slider for z slices
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
minval = np.min(resampled_data_array[:,:,:,0])
maxval = np.max(resampled_data_array[:,:,:,0])
def f(z):
    plt.imshow(np.flip(resampled_data_array,axis=0)[:,:,z,0].T, cmap='gray', vmin=minval, vmax=maxval)
    plt.colorbar()
    plt.show()

interact(f, z=widgets.IntSlider(min=0,max=resampled_data_array.shape[2]-1,step=1,value=0))

def f(z):
    plt.imshow(resampled_data_array[:,:,z,0].T, cmap='gray', vmin=minval, vmax=maxval)
    plt.colorbar()
    plt.show()

interact(f, z=widgets.IntSlider(min=0,max=resampled_data_array.shape[2]-1,step=1,value=0))

interactive(children=(IntSlider(value=0, description='z', max=220), Output()), _dom_classes=('widget-interact'…

interactive(children=(IntSlider(value=0, description='z', max=220), Output()), _dom_classes=('widget-interact'…

<function __main__.f(z)>

In [38]:
maxval

65535

: 